**Setup:**

In [13]:
%pip install -Uqq fastai torchvision pickle

from fastai.vision.all import *

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Note: you may need to restart the kernel to use updated packages.


Importing Data:

In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

Look at `train_data`:

In [3]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
train_data.data.shape

torch.Size([60000, 28, 28])

Create Dataloaders `dl`, `valid_dl` and `dls`:

In [5]:
dl = DataLoader(train_data, batch_size=100, shuffle=True)
valid_dl = DataLoader(test_data, batch_size=100, shuffle=True)
dls = DataLoaders(dl, valid_dl)

Import `Net` from `learning_functions.py`:

In [6]:
from learning_functions import Net
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Use GPU if available
neural_net = Net().to(device)

Create `learn` function from `neural_net`:

In [7]:
from torch import nn
from learning_functions import batch_accuracy

learn = Learner(dls, neural_net, opt_func=SGD, loss_func=nn.CrossEntropyLoss(), metrics=batch_accuracy)

In [8]:
optimizer = SGD(neural_net.parameters(), lr=0.2)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    neural_net.train()
    global optimizer
    for batch_idx, (data, target) in enumerate(dl):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = neural_net(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx*len(data)}/{len(dl.dataset)} ({100. * batch_idx / len(dl):.0f}%)]\t{loss.item():.6f}")

def test():
    neural_net.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in valid_dl:
            data, target = data.to(device), target.to(device)
            output = neural_net(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(valid_dl.dataset)
    print(f'\nTest Loss: Average Loss: {test_loss:.4f}, Accuracy: {correct}/{len(valid_dl.dataset)} {100. * correct / len(valid_dl.dataset):.0f}%')

In [9]:
from torch import optim
# from learning_functions import train, test

for epoch in range(1, 11):
        train(epoch)
        test()

/Users/preston/Projects/fastai_course/MNIST_Digit_identifier/learning_functions.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	2.302407
Train Epoch: 1 [2000/60000 (3%)]	2.302410
Train Epoch: 1 [4000/60000 (7%)]	2.301476
Train Epoch: 1 [6000/60000 (10%)]	2.300585
Train Epoch: 1 [8000/60000 (13%)]	2.299006
Train Epoch: 1 [10000/60000 (17%)]	2.297978
Train Epoch: 1 [12000/60000 (20%)]	2.295551
Train Epoch: 1 [14000/60000 (23%)]	2.290289
Train Epoch: 1 [16000/60000 (27%)]	2.291070
Train Epoch: 1 [18000/60000 (30%)]	2.242637
Train Epoch: 1 [20000/60000 (33%)]	2.222497
Train Epoch: 1 [22000/60000 (37%)]	2.127462
Train Epoch: 1 [24000/60000 (40%)]	2.014867
Train Epoch: 1 [26000/60000 (43%)]	1.943990
Train Epoch: 1 [28000/60000 (47%)]	1.970862
Train Epoch: 1 [30000/60000 (50%)]	1.835616
Train Epoch: 1 [32000/60000 (53%)]	1.796894
Train Epoch: 1 [34000/60000 (57%)]	1.809688
Train Epoch: 1 [36000/60000 (60%)]	1.802201
Train Epoch: 1 [38000/60000 (63%)]	1.845780
Train Epoch: 1 [40000/60000 (67%)]	1.720463
Train Epoch: 1 [42000/60000 (70%)]	1.787964
Train Epoch: 1 [44000/60000 (73%)]	1.733008

In [11]:
learn = Learner(dls, neural_net, opt_func=SGD, loss_func=loss_fn)
learn.fit(5, 0.2)

epoch,train_loss,valid_loss,time
0,1.540924,1.492586,00:12
1,1.538203,1.491720,00:12
2,1.534481,1.489440,00:11
3,1.527587,1.486569,00:12
4,1.529370,1.486071,00:12


/Users/preston/Projects/fastai_course/MNIST_Digit_identifier/learning_functions.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


In [14]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

save_object(neural_net, 'digit_identifier.pkl')